In [1]:
# Imports
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split


In [2]:
df = pd.read_csv('./data/processed_data.csv')
df.shape


(18471, 69)

In [3]:
# Final labels
labels = ['patientState_good', 'patientState_caution', 'patientState_danger']
# Section labels
section_labels = {
    'fever': ['feverState_good', 'feverState_caution', 'feverState_danger'],
    'medication': ['medicationState_good', 'medicationState_caution', 'medicationState_danger'],
    'hydration': ['hydrationState_good', 'hydrationState_caution', 'hydrationState_danger'],
    'respiration': ['respirationState_good', 'respirationState_caution', 'respirationState_danger'],
    'skin': ['skinState_good', 'skinState_caution', 'skinState_danger'],
    'pulse': ['pulseState_good', 'pulseState_caution', 'pulseState_danger'],
    'general': ['generalState_good', 'generalState_caution', 'generalState_danger'],
}
# Section labels in an array flattened
section_labels_arr = [item for val in section_labels.values() for item in val]
# All labels
all_labels = [*labels, *section_labels_arr]
len(all_labels)


24

In [4]:
df[section_labels['general']].value_counts()


generalState_good  generalState_caution  generalState_danger
1.0                0.0                   0.0                    10142
0.0                1.0                   0.0                     7976
                   0.0                   1.0                      353
dtype: int64

In [5]:
# Separating to testing and training

# Prepare the data: You should split your data into training and test sets.
# The training set will be used to train the model and the test set will be used
# to evaluate the model's performance.

_x, x_test, _y, y_test = train_test_split(
    df.drop(columns=all_labels).to_numpy(),
    df[labels].to_numpy(),
    test_size=0.2,
    random_state=42
)
x_train, x_val, y_train, y_val = train_test_split(
    _x,
    _y,
    test_size=0.25,
    train_size=0.75,
    random_state=42
)
x_train.shape


(11082, 45)

In [6]:
str(list(x_test[1]))


'[15.6173706445342, 1.0, 38.3, 0.0, 0.0, 0.0, 1.0, 1.0, 50.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 13.0, 0.0, 0.0, 0.0, 0.0, 68.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]'

In [7]:
# Preprocess the data: Data preprocessing is an important step, which includes
# cleaning and transforming the data. You should normalize the data,
# one-hot encode categorical variables, and split the data into features and labels.

# TODO: Does KNNImputer create a correlation between training and test split?
# If yes do ->
# TODO: move KNNImputer here
# TODO: move every preprocessing step that would create a connection between train and test split


In [8]:
from tensorflow import keras


In [9]:
# Create a baseline model to measure performance
b_model = keras.Sequential([
    keras.layers.Dense(units=45, activation='tanh', input_shape=(45,)),
    keras.layers.Dense(units=3, activation='softmax'),
])
b_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
b_model.build()
b_model.summary()


Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 45)                2070      
                                                                 
 dense_1 (Dense)             (None, 3)                 138       
                                                                 
Total params: 2,208
Trainable params: 2,208
Non-trainable params: 0
_________________________________________________________________


2023-02-25 16:04:28.734411: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-25 16:04:28.734510: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [10]:
# Train baseline model
NUM_EPOCHS = 20

# Early stopping set after 5 epochs
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

b_model.fit(
    x_train,
    y_train,
    epochs=NUM_EPOCHS,
    validation_data=(x_val, y_val),
    callbacks=[stop_early],
    verbose=2
)

Epoch 1/20


2023-02-25 16:04:28.983892: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-25 16:04:29.130242: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-02-25 16:04:31.415671: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


347/347 - 3s - loss: 0.7544 - accuracy: 0.6608 - val_loss: 0.6614 - val_accuracy: 0.7044 - 3s/epoch - 9ms/step
Epoch 2/20
347/347 - 3s - loss: 0.6018 - accuracy: 0.7712 - val_loss: 0.5533 - val_accuracy: 0.7975 - 3s/epoch - 8ms/step
Epoch 3/20
347/347 - 3s - loss: 0.5187 - accuracy: 0.8079 - val_loss: 0.4975 - val_accuracy: 0.8059 - 3s/epoch - 8ms/step
Epoch 4/20
347/347 - 3s - loss: 0.4709 - accuracy: 0.8252 - val_loss: 0.4654 - val_accuracy: 0.8249 - 3s/epoch - 8ms/step
Epoch 5/20
347/347 - 3s - loss: 0.4436 - accuracy: 0.8319 - val_loss: 0.4485 - val_accuracy: 0.8170 - 3s/epoch - 8ms/step
Epoch 6/20
347/347 - 3s - loss: 0.4191 - accuracy: 0.8396 - val_loss: 0.4235 - val_accuracy: 0.8368 - 3s/epoch - 8ms/step
Epoch 7/20
347/347 - 3s - loss: 0.4075 - accuracy: 0.8429 - val_loss: 0.4158 - val_accuracy: 0.8414 - 3s/epoch - 8ms/step
Epoch 8/20
347/347 - 3s - loss: 0.3925 - accuracy: 0.8466 - val_loss: 0.3979 - val_accuracy: 0.8457 - 3s/epoch - 9ms/step
Epoch 9/20
347/347 - 3s - loss: 0.3

In [11]:
def evaluate_model(model, x_test, y_test):
    """
    evaluate model on test set and show results in dataframe.
    
    Parameters
    ----------
    model : keras model
        trained keras model.
    X_test : numpy array
        Features of holdout set.
    y_test : numpy array
        Labels of holdout set.
        
    Returns
    -------
    display_df : DataFrame
        Pandas dataframe containing evaluation results.
    """
    eval_dict = model.evaluate(x_test, y_test, return_dict=True)
    
    display_df = pd.DataFrame([eval_dict.values()], columns=[list(eval_dict.keys())])
    
    return display_df

# Evaluate model on test set and add results to dataframe
results = evaluate_model(b_model, x_test, y_test)

# Set index to 'Baseline'
results.index = ['Baseline']

# Display results
results.head()

116/116 [==============================] - 1s 7ms/step - loss: 0.3167 - accuracy: 0.8788


,loss,accuracy
Baseline,0.316732,0.878755


In [12]:
# Define the model: TensorFlow provides a high-level API for building and
# training neural network models. You should choose the appropriate model
# architecture for your problem and specify the hyperparameters,
# such as the number of hidden layers and the number of neurons in each layer.

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath='./checkpoints/model_{epoch}',
        save_freq='epoch'),
    keras.callbacks.TensorBoard(log_dir='./logs')
]


def build_model(hp):
    """
    Builds model and sets up hyperparameter space to search.
    
    Parameters
    ----------
    hp : HyperParameter object
        Configures hyperparameters to tune.
        
    Returns
    -------
    model : keras model
        Compiled model with hyperparameters to tune.
    """
    model = keras.Sequential([
        keras.layers.Dense(
            units=hp.Int('units', min_value=32, max_value=512, step=32),
            activation=hp.Choice('activation', values=['tanh', 'sigmoid', 'relu']),
            input_shape=(45,)),
    ])
    # Tune the number of hidden layers and units in each.
    # Number of hidden layers: 1 - 4
    # Number of Units: 32 - 512 with stepsize of 32
    num_layers = hp.Int("num_layers", 2, 5)
    for i in range(1, num_layers):
        with hp.conditional_scope('num_layers', list(range(i+1, 5+1))):
            model.add(
                keras.layers.Dense(
                    units=hp.Int("units_" + str(i), min_value=32, max_value=512, step=32),
                    activation=hp.Choice('activation_' + str(i), values=['tanh', 'sigmoid', 'relu']),
                )
            )
            # Tune dropout layer with values from 0 - 0.3 with stepsize of 0.1.
            model.add(keras.layers.Dropout(hp.Float("dropout_" + str(i), 0, 0.3, step=0.1)))

    # Output layer
    model.add(
        keras.layers.Dense(3, activation='softmax')
    )
    model.compile(
        optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        # optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    model.build()
    return model


In [13]:
# Compile the model: You should compile the model by specifying the optimizer,
# loss function, and evaluation metrics.
import keras_tuner

tuner = keras_tuner.tuners.Hyperband(
    build_model,
    objective='val_loss',
    executions_per_trial=2,
    hyperband_iterations=2,
    max_epochs=100,
    factor=3,
    overwrite=False,
    directory='tuner3',
    project_name="hyperband",
)


INFO:tensorflow:Reloading Oracle from existing project tuner3/hyperband/oracle.json
INFO:tensorflow:Reloading Tuner from tuner3/hyperband/tuner0.json


In [14]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tuner.search(x_train, y_train, validation_data=(x_val, y_val), epochs=4, callbacks=[stop_early])


Trial 240 Complete [00h 01m 44s]
val_loss: 0.6574364900588989

Best val_loss So Far: 0.21755419671535492
Total elapsed time: 00h 05m 03s

Search: Running Trial #241

Value             |Best Value So Far |Hyperparameter
96                |256               |units
tanh              |sigmoid           |activation
5                 |3                 |num_layers
320               |256               |units_1
sigmoid           |relu              |activation_1
0.2               |0.1               |dropout_1
0.0001            |0.001             |learning_rate
256               |416               |units_2
sigmoid           |sigmoid           |activation_2
0.2               |0.1               |dropout_2
384               |None              |units_3
tanh              |None              |activation_3
0                 |None              |dropout_3
384               |None              |units_4
relu              |None              |activation_4
0.2               |None              |dropout_4
34     

2023-02-25 16:10:30.124446: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


347/347 [==============================] - ETA: 0s - loss: 0.8150 - accuracy: 0.6624

2023-02-25 16:10:35.167672: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


347/347 [==============================] - 6s 17ms/step - loss: 0.8150 - accuracy: 0.6624 - val_loss: 0.7519 - val_accuracy: 0.6600
Epoch 2/34
347/347 [==============================] - 5s 13ms/step - loss: 0.7174 - accuracy: 0.6803 - val_loss: 0.6668 - val_accuracy: 0.7293
Epoch 3/34
347/347 [==============================] - 5s 13ms/step - loss: 0.6427 - accuracy: 0.7364 - val_loss: 0.5941 - val_accuracy: 0.7699
Epoch 4/34
347/347 [==============================] - 5s 13ms/step - loss: 0.5673 - accuracy: 0.7790 - val_loss: 0.5465 - val_accuracy: 0.7799
Epoch 5/34
347/347 [==============================] - 5s 13ms/step - loss: 0.5295 - accuracy: 0.7895 - val_loss: 0.5095 - val_accuracy: 0.7937
Epoch 6/34
347/347 [==============================] - 5s 13ms/step - loss: 0.4962 - accuracy: 0.8064 - val_loss: 0.4852 - val_accuracy: 0.8116
Epoch 7/34
347/347 [==============================] - 5s 13ms/step - loss: 0.4802 - accuracy: 0.8136 - val_loss: 0.4986 - val_accuracy: 0.7953
Epoch 8/34

2023-02-25 16:13:09.062839: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


347/347 [==============================] - ETA: 0s - loss: 0.8161 - accuracy: 0.6626

2023-02-25 16:13:13.200942: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


347/347 [==============================] - 5s 14ms/step - loss: 0.8161 - accuracy: 0.6626 - val_loss: 0.7373 - val_accuracy: 0.6724
Epoch 2/34
157/347 [============>.................] - ETA: 41s - loss: 0.7489 - accuracy: 0.6636

In [ ]:

tuner.results_summary()


In [ ]:
tuner.search_space_summary()


Best val_loss So Far: 0.3011022210121155

- 5 epochs
- 512 units
- tanh activation
- True 2nd layer
- 0.001 learning_rate
- 192 units2


In [ ]:
results

In [ ]:
params = tuner.get_best_hyperparameters(10)

for i, param in enumerate(params):
    print('Running model:', i)
    model = tuner.hypermodel.build(param)
    model.fit(x_train, y_train, epochs=param.values.get('tuner/epochs'), callbacks=callbacks, validation_data=(x_val, y_val))
    hyper_df = evaluate_model(model, x_test, y_test)
    hyper_df.index = ["Hypertuned-" + str(i)]
    hyper_df['units'] = param.values.get('units')
    hyper_df['activation'] = param.values.get('activation')
    hyper_df['learning_rate'] = param.values.get('learning_rate')
    hyper_df['epochs'] = param.values.get('tuner/epochs')
    hyper_df['2nd_layer'] = param.values.get('2nd_layer')
    hyper_df['units2'] = param.values.get('units2')
    # Append results in dataframe
    results = pd.concat([results, hyper_df])

Model 1: 116/116 [==============================] - 3s 24ms/step - loss: 0.2007 - accuracy: 0.9369


In [ ]:
results

In [ ]:
model = build_model(params)


In [ ]:
model.fit(x_train, y_train, callbacks=callbacks, epochs=25, validation_data=(x_val, y_val))


In [ ]:
# Evaluate the model: You should evaluate the performance of the model
# on the test set and compare it to the training set performance to determine
# if the model has overfitted or underfitted the data.
loss, acc = model.evaluate(x_test, y_test)

print("loss: %.2f" % loss)
print("acc: %.2f" % acc)


In [ ]:
r = model.predict(x_test)
# res = []
# tmp = []
# for i in range(24):
#   if i%3 ==0 and i !=0:
#     res.append(tmp)
#     tmp = []
#   tmp.append(r[i])
# res.append(tmp)
# res
res = [np.argmax(i) for i in r]


In [ ]:
model.save('./out/model')


In [ ]:
# Convert to TFLite model
import tensorflow as tf

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(
    './out/model')  # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)
